# Load Data

In [1]:
# Imports
# from settings import *
# import analyze_cascade
import datetime
import pandas as pd
import numpy as np

In [2]:
metadata_file = 'metadata_anon.txt'

In [3]:
# Read meta data 
fin = open(metadata_file,'r')
lines = fin.readlines()
fin.close()
cascade_id2metadata={}
for line in lines:
    line = line.replace('\n','')
    item = eval(line)
    cascade_id2metadata[item[0]] = item[1]

## Data frame of static measures

In [4]:
# Get static measures
veracity = []
virality = []
depth = []
breadth = []
size = []
verified = []
nfollowers = []
nfollowees = []
engagement = []
category = []
day = []
hour = []
for cascade,metadata in cascade_id2metadata.items():
    veracity.append(metadata['veracity'])
    virality.append(metadata['virality'])
    depth.append(metadata['depth'])
    breadth.append(metadata['max_breadth'])
    size.append(metadata['size'])
    verified.append(metadata['verified_list'][0])
    nfollowers.append(metadata['num_followers_list'][0])
    nfollowees.append(metadata['num_followees_list'][0])
    engagement.append(metadata['engagement_list'][0])
    category.append(metadata['rumor_category'])
    day.append(metadata['start_date'].day)
    hour.append(metadata['start_date'].hour)

# Convert to data frame
df = pd.DataFrame({'veracity': veracity, 'virality': virality, 'depth': depth, 'breadth': breadth, 'size': size, 'verified': verified, 'nfollowers': nfollowers, 
                   'nfollowees': nfollowees, 'engangement': engagement, 'category': category, 'day': day, 'hour': hour})

# Inspect
df.head(5)


,breadth,category,day,depth,engangement,hour,nfollowees,nfollowers,size,veracity,verified,virality
0,1,Politics,22,0,7.092405,21,236.0,164.0,1,FALSE,False,NaN
1,1,Politics,23,0,8.394500,14,1999.0,870.0,1,FALSE,False,NaN
2,1,Politics,9,0,6.729679,16,1982.0,452.0,1,FALSE,False,NaN
3,1,Politics,15,0,0.186418,20,55.0,5.0,1,FALSE,False,NaN
4,1,Politics,14,0,1.425915,14,129.0,95.0,1,FALSE,False,NaN
